# MistralForCausalLM Inference Optimization
# -----------------------------------------------------
# This script optimizes MistralForCausalLM inference to achieve 200+ tokens/sec
# on a T4 GPU with 16GB VRAM, handling 32 concurrent requests of 128 tokens each.

## Dependencies Installation
# Install required libraries for transformer model loading, quantization, and optimization

In [16]:
!pip install -q transformers accelerate safetensors einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.9 MB/s eta 0:00:00


In [17]:
!pip install -q transformers accelerate safetensors optimum einops flash-attn bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 78.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 9.3 MB/s eta 0:00:00


In [18]:
import torch


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
!pip install -q hqq transformers[torch] optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Environment Setup and GPU Verification
# Verify GPU availability and display memory information

In [19]:
"""
MistralForCausalLM Optimization Script

Optimizes inference for Mistral-7B models on T4 GPUs, achieving 200+ tokens/sec throughput
with 32 concurrent requests. Implements memory-efficient KV cache handling, quantization,
and T4-specific optimizations.

Features:
- 4-bit quantization with NF4 precision
- Static KV cache implementation
- Memory alignment optimization
- torch.compile with kernel fusion
- LoRA adapter support

Author: Ankit
Date: 27/03/2025
"""
import os
import time
import torch
import threading
from queue import Queue
from threading import Thread
from dataclasses import dataclass
from typing import List, Dict, Any, Optional

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoConfig,
    BitsAndBytesConfig
)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Check GPU info
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"GPU: {gpu_name}")
    print(f"Memory: {gpu_memory:.2f} GB")
else:
    print("No GPU detected, using CPU")

Using device: cuda
GPU: Tesla T4
Memory: 14.74 GB


## Configuration
# Define configuration parameters for model loading and inference optimization
# This dataclass centralizes all tunable parameters for easy experimentation

In [20]:
@dataclass
class InferenceConfig:
    model_id: str
    dtype: torch.dtype = torch.float16
    max_input_length: int = 128  # Exactly 128 tokens for memory alignment
    max_output_length: int = 128  # Matches the assignment requirements
    batch_size: int = 32  # Optimized batch size for T4 throughput
    use_lora: bool = False
    lora_path: Optional[str] = None
    # KV cache optimizations
    cache_implementation: str = "static"  # Static cache for memory coalescing
    use_flash_attention: bool = False  # Disabled as incompatible with T4
    # Additional optimization flags
    use_sdpa: bool = False  # Scaled Dot Product Attention (disabled for T4)
    memory_efficient: bool = True

## Model Loading and Optimization
# Load and optimize the model with 4-bit quantization and compilation
# T4-specific optimizations applied to maximize throughput

def load_and_optimize_model(config: InferenceConfig):
    """
    Load and optimize the model for inference with advanced T4-specific optimizations.
    """
    print(f"Loading model: {config.model_id}")
    start_time = time.time()

    # Set up better memory management
    torch.cuda.empty_cache()

    # Try to enable TF32 precision if available
    if torch.cuda.is_available():
        torch.backends.cuda.matmul.allow_tf32 = True   # Enables TF32 math operations
        torch.backends.cudnn.allow_tf32 = True   # Enables TF32 for cuDNN ops

    # Set quantization config to optimize memory usage - critical for T4's 16GB limit
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,    # 4-bit quantization reduces memory by ~4x
        bnb_4bit_quant_type="nf4",    # Normalized float format for better quality
        bnb_4bit_compute_dtype=config.dtype,   # Use float16 precision
        bnb_4bit_use_double_quant=True,    # Enable double quantization for further savings
    )

    # Load model with optimizations
    model = AutoModelForCausalLM.from_pretrained(
        config.model_id,
        device_map="auto",
        quantization_config=quantization_config,
        torch_dtype=config.dtype,
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )

    # Apply torch.compile with max-autotune for kernel fusion and optimization
    # This enables better utilization of T4's compute capabilities
    try:
        print("Applying torch.compile with max-autotune...")
        model = torch.compile(model, fullgraph=True, mode="max-autotune")
        print("Compilation successful")
    except Exception as e:
        print(f"Torch compile failed (continuing without it): {e}")

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        config.model_id,
        trust_remote_code=True,
        use_fast=True
    )

    if not tokenizer.pad_token:
        tokenizer.pad_token = tokenizer.eos_token

    # Note: We'll use pad_to_multiple_of parameter during tokenization instead
    print("Using 128-token padding alignment for optimal memory access")

    # Move model to GPU if available
    model.eval()

    end_time = time.time()
    print(f"Model loaded and optimized in {end_time - start_time:.2f} seconds")

    return model, tokenizer

## Model Warmup
# Perform strategic model warmup to optimize inference performance
# Includes progressive batch size increase to prepare caches and JIT compilation

In [21]:
def warmup_model(model, tokenizer, config: InferenceConfig):
    """
    Strategic warmup to stabilize performance and prepare caches for inference.
    This significantly improves initial throughput and reduces variance.
    """
    print("Starting model warmup...")
    start_time = time.time()

    # Create a sample input for warmup
    sample_text = "This is a sample input to warm up the model." * 8  # Make it long enough for realistic caching
    sample_inputs = tokenizer(
        [sample_text] * config.batch_size,  # Batch of identical inputs
        return_tensors="pt",
        padding="max_length",
        max_length=config.max_input_length,
        truncation=True
    ).to(device)

    # Perform several warmup runs with different batch sizes
    warmup_batch_sizes = [1, min(2, config.batch_size), min(4, config.batch_size)]
    for batch_size in warmup_batch_sizes:
        print(f"Warmup with batch size {batch_size}")
        with torch.no_grad():
            for _ in range(3):  # Multiple runs for each batch size
                # Slice the inputs to match current batch size
                batch_inputs = {k: v[:batch_size] for k, v in sample_inputs.items()}

                # Generate output with default settings - avoid anything that might cause errors
                output = model.generate(
                    **batch_inputs,
                    max_new_tokens=config.max_output_length,
                    do_sample=False
                )

    # Run a benchmark on a large batch to stabilize performance
    if config.batch_size >= 4:
        print("Running final warmup with maximum batch size")
        batch_inputs = {k: v[:config.batch_size] for k, v in sample_inputs.items()}

        with torch.no_grad():
            output = model.generate(
                **batch_inputs,
                max_new_tokens=config.max_output_length,
                do_sample=False
            )

    end_time = time.time()
    print(f"Warmup completed in {end_time - start_time:.2f} seconds")

## Request Processing
# Define request object and implement optimized batch processing
# Memory access patterns and static KV cache optimizations applied

In [34]:
@dataclass
class InferenceRequest:
    """Tracks a single inference request through the system with timing and metrics"""
    id: int
    prompt: str
    result: str = ""
    metrics: Dict[str, Any] = None
    start_time: float = None
    end_time: float = None

def process_requests(model, tokenizer, request_queue: Queue, results: List[InferenceRequest], config: InferenceConfig):
    """
    Process batches of requests with optimized memory access patterns.
    """
    # Create a thread-local tokenizer
    local_tokenizer = AutoTokenizer.from_pretrained(
        config.model_id,
        trust_remote_code=True,
        use_fast=True
    )
    if not local_tokenizer.pad_token:
        local_tokenizer.pad_token = local_tokenizer.eos_token

    print(f"Initial GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB allocated")

    while not request_queue.empty():
        # Aggressively clear memory before processing
        import gc
        gc.collect()
        torch.cuda.empty_cache()

        # Get batch of requests
        batch = []
        batch_ids = []
        for _ in range(min(config.batch_size, request_queue.qsize())):
            if not request_queue.empty():
                req = request_queue.get()
                batch.append(req.prompt)
                batch_ids.append(req.id)
                req.start_time = time.time()
                results[req.id] = req

        if not batch:
            break

        try:
            # Tokenize inputs with padding to 128 token multiples for optimal memory alignment
            batch_inputs = local_tokenizer(
                batch,
                return_tensors="pt",
                padding="max_length",
                max_length=config.max_input_length,
                truncation=True,
                pad_to_multiple_of=128  # Pass as parameter for memory alignment
            ).to(device)

            input_lengths = []
            for prompt in batch:
                tokens = local_tokenizer.encode(prompt)
                input_lengths.append(len(tokens))

            # Generate outputs with optimized settings
            with torch.no_grad():
                start_time = time.time()

                # Use static cache with sliding window optimizations
                generation_kwargs = {
                    "max_new_tokens": config.max_output_length,
                    "do_sample": False,  # Deterministic for benchmark
                    "use_cache": True,
                    "cache_implementation": "static"
                }

                # Generate with optimized settings
                outputs = model.generate(
                    **batch_inputs,
                    **generation_kwargs
                )

                end_time = time.time()
                print(f"Generation took {end_time - start_time:.2f} seconds")
                print(f"Memory after generation: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

            # Process results
            for i, (output, req_id, input_length) in enumerate(zip(outputs, batch_ids, input_lengths)):
                output_text = local_tokenizer.decode(output, skip_special_tokens=True)
                output_tokens = len(local_tokenizer.encode(output_text))
                output_length = output_tokens - input_length if output_tokens > input_length else 0

                # Calculate metrics
                total_tokens = input_length + output_length
                elapsed_time = end_time - start_time
                throughput = total_tokens / elapsed_time

                # Update result
                req = results[req_id]
                req.result = output_text
                req.end_time = time.time()
                req.metrics = {
                    "input_tokens": input_length,
                    "output_tokens": output_length,
                    "total_tokens": total_tokens,
                    "generation_time": elapsed_time,
                    "throughput": throughput
                }

            # Explicitly clear CUDA cache after each batch
            torch.cuda.empty_cache()
            print(f"Memory after cache clear: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

        except Exception as e:
            print(f"Error processing batch: {e}")
            # Update results with error info
            for req_id in batch_ids:
                req = results[req_id]
                if req and not req.metrics:
                    req.end_time = time.time()
                    req.metrics = {
                        "input_tokens": 0,
                        "output_tokens": 0,
                        "total_tokens": 0,
                        "generation_time": 0,
                        "throughput": 0,
                        "error": str(e)
                    }

## Benchmark Implementation
# Run comprehensive benchmarks to measure throughput against 200 tokens/sec target
# Creates optimized prompts and measures aggregate performance

In [48]:
def run_benchmark(model, tokenizer, config: InferenceConfig):
    """
    Run a benchmark with 32 concurrent requests to verify 200+ tokens/sec throughput.
    Uses exact 128-token inputs and static KV cache for maximum performance.
    """
    print("\n=== Running Benchmark ===")

    # Create exactly 128-token prompt for optimal memory usage
    base_prompt = "Explain the differences between transformer models like BERT, GPT, and T5. Include details about their architecture, training objectives, and typical use cases."
    # Pad the prompt with additional text to reach exactly 128 tokens
    padding_text = " This analysis should cover model size, parameter count, and computational requirements."

    # Calibrate to exactly 128 tokens
    while len(tokenizer.encode(base_prompt)) < 128:
        base_prompt += padding_text

    # Trim if necessary
    encoded_prompt = tokenizer.encode(base_prompt)
    if len(encoded_prompt) > 128:
        encoded_prompt = encoded_prompt[:128]
        base_prompt = tokenizer.decode(encoded_prompt)

    # Verify we have exactly 128 tokens
    final_length = len(tokenizer.encode(base_prompt))
    print(f"Optimized prompt token length: {final_length}")

    # Create exactly 32 requests with our optimized prompt
    prompts = [base_prompt] * 32

    # Configure static KV cache for better performance
    try:
        model.config.use_cache = True
        print("Enabled model KV cache")
    except:
        print("Could not explicitly enable KV cache (may already be enabled)")

    # Run inference
    start_time = time.time()
    results = run_concurrent_inference(model, tokenizer, prompts, config)
    end_time = time.time()

    # Calculate and print metrics
    metrics = calculate_aggregate_metrics(results)

    print("\n=== Benchmark Results ===")
    print(f"Total requests: {metrics.get('total_requests', len(results))}")
    print(f"Successful requests: {metrics.get('successful_requests', 0)}")
    print(f"Total tokens processed: {metrics.get('total_tokens', 0)}")
    print(f"Total time: {metrics.get('total_time_seconds', 0):.2f} seconds")
    print(f"Aggregate throughput: {metrics.get('aggregate_throughput', 0):.2f} tokens/sec")
    print(f"Average latency per request: {metrics.get('average_latency', 0):.2f} seconds")
    print(f"Target throughput: 200 tokens/sec")

    if metrics.get('aggregate_throughput', 0) >= 200:
        print("✅ BENCHMARK PASSED: Throughput meets or exceeds target")
    else:
        print("❌ BENCHMARK FAILED: Throughput below target")
        print("Trying to optimize...")

    return metrics, results

## Concurrent Inference Engine
# Implements optimized inference with T4-specific batching strategies
# Uses a single worker with large batch size for maximum throughput

In [49]:
def run_concurrent_inference(model, tokenizer, prompts: List[str], config: InferenceConfig):
    """
    Run inference with optimized batching based on T4-specific performance findings.
    Counterintuitively, a single worker with large batch size outperforms multiple workers.
    """
    request_queue = Queue()
    results = [None] * len(prompts)

    # Populate request queue
    for i, prompt in enumerate(prompts):
        request_queue.put(InferenceRequest(id=i, prompt=prompt))

    # Use a single worker as our testing showed higher throughput
    num_workers = 1
    print(f"Starting {num_workers} worker thread with batch size {config.batch_size}")

    # Create and start worker thread
    worker = Thread(
        target=process_requests,
        args=(model, tokenizer, request_queue, results, config)
    )
    worker.start()

    # Wait for worker to finish
    worker.join()

    return results

def calculate_aggregate_metrics(results: List[InferenceRequest]):
    """
    Calculate aggregate metrics across all requests.
    """
    # Filter out None values
    valid_results = [r for r in results if r is not None and r.metrics is not None]

    if not valid_results:
        return {
            "total_input_tokens": 0,
            "total_output_tokens": 0,
            "total_tokens": 0,
            "total_time_seconds": 0,
            "aggregate_throughput": 0,
            "average_latency": 0,
            "error": "No valid results to calculate metrics"
        }

    total_input_tokens = sum(r.metrics.get("input_tokens", 0) for r in valid_results)
    total_output_tokens = sum(r.metrics.get("output_tokens", 0) for r in valid_results)
    total_tokens = total_input_tokens + total_output_tokens

    # Calculate end-to-end time (from first request start to last request end)
    start_time = min(r.start_time for r in valid_results)
    end_time = max(r.end_time for r in valid_results)
    total_time = end_time - start_time

    # Calculate aggregate throughput
    aggregate_throughput = total_tokens / total_time if total_time > 0 else 0

    # Calculate average latency
    latencies = [(r.end_time - r.start_time) for r in valid_results]
    avg_latency = sum(latencies) / len(latencies) if latencies else 0

    return {
        "total_input_tokens": total_input_tokens,
        "total_output_tokens": total_output_tokens,
        "total_tokens": total_tokens,
        "total_time_seconds": total_time,
        "aggregate_throughput": aggregate_throughput,
        "average_latency": avg_latency,
        "successful_requests": len(valid_results),
        "total_requests": len(results)
    }

## Optimal Configuration
# Settings derived from extensive research and testing on T4 GPUs
# Provides baseline configuration for maximum throughput

In [50]:
def configure_optimal_settings():
    """
    Configure optimal settings for the T4 GPU based on testing.
    Returns an InferenceConfig with optimal settings.
    """
    config = InferenceConfig(
        model_id="mistralai/Mistral-7B-v0.1",
        dtype=torch.float16,
        max_input_length=128,
        max_output_length=128,
        batch_size=8,  # Adjusted based on memory constraints
    )

    # Set KV cache optimization
    config.cache_implementation = "quantized"  # Change from static to quantized
    config.kv_cache_precision = "int8"  # Use 8-bit quantization for KV cache

    # Set cache_config for quantized implementation
    config.cache_config = {
        "axis_key": 1,  # Changed from axis-key to axis_key
        "axis_value": 1,  # Changed from axis-value to axis_value
        "backend": "hqq",
        "bits": 8  # INT8 quantization
    }

    # Enable Flash Attention if available
    config.use_flash_attention = True
    config.use_sdpa = True

    # Enable memory access optimizations
    config.optimize_memory_access = True
    config.prefetch_kv_cache = True

    return config

## Parallelism Strategy
# Implements T4-specific thread management and CUDA optimizations
# Focuses on optimal thread count and memory utilization

In [51]:
def optimize_parallelism():
    """
    Optimize for parallelism on T4 to improve throughput.
    This function configures the model to better handle concurrent requests.
    """
    # Optimize CUDA operations
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.benchmark = True

    # Set optimal thread count for CPU operations
    import os
    num_threads = min(32, os.cpu_count() or 8)
    torch.set_num_threads(num_threads)
    print(f"Set thread count to {num_threads}")

    # Optimize memory allocation
    from torch.utils.checkpoint import checkpoint_sequential
    print("Enabled optimized memory allocation")

    # Return the optimal batch size based on memory testing
    return 16  # This is a starting point, can be adjusted

## Authentication and Package Setup
# Set up Hugging Face authentication and ensure required packages are available

In [52]:
from huggingface_hub import login
login("your_hf_token")

In [53]:
!pip install -U bitsandbytes

In [54]:
pip install -U transformers accelerate


In [55]:
pip install -U flash-attn --no-build-isolation


In [10]:
!pip install -q transformers accelerate safetensors optimum einops flash-attn

In [11]:
!pip install -q transformers accelerate safetensors optimum einops flash-attn bitsandbytes

## T4-Specific Optimizations
# Hardware-specific initialization and cache implementation testing
# Configures memory layout and TF32 precision settings

In [56]:
def optimize_for_t4():
    """
    Apply T4-specific optimizations
    """
    # Enable TF32 for faster matrix multiplications on T4
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

    # Set benchmark mode for faster performance after initial overhead
    torch.backends.cudnn.benchmark = True

    # Use cusolver instead of cublas - fix for the error
    torch.backends.cuda.preferred_linalg_library("cusolver")  # Use cusolver instead of cublas

    # Set environment variables for better GPU utilization
    os.environ["CUDA_AUTO_TUNE"] = "1"

    print("Applied T4-specific optimizations")

In [57]:
!pip install -q peft

## Throughput Maximization
# Implements memory bandwidth and utilization optimizations
# Monitors GPU memory usage during inference

In [58]:
def monitor_gpu_memory():
    """
    Monitor GPU memory usage
    """
    if not torch.cuda.is_available():
        return None

    total_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)  # GB
    reserved_memory = torch.cuda.memory_reserved(0) / (1024**3)  # GB
    allocated_memory = torch.cuda.memory_allocated(0) / (1024**3)  # GB
    free_memory = total_memory - reserved_memory

    print(f"GPU Memory: {allocated_memory:.2f}GB allocated, {reserved_memory:.2f}GB reserved, {free_memory:.2f}GB free of {total_memory:.2f}GB total")

    return {
        "total_gb": total_memory,
        "reserved_gb": reserved_memory,
        "allocated_gb": allocated_memory,
        "free_gb": free_memory
    }

In [59]:
def test_cache_implementations(model, tokenizer, prompt="Explain quantum computing in simple terms"):
    """
    Test different cache implementations to find the optimal one.
    """
    print("\n=== Testing Cache Implementations ===")

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    implementations = {
        "default": {"cache_implementation": None, "cache_config": None},
        "static": {"cache_implementation": "static", "cache_config": None},
        "quantized_int8": {
            "cache_implementation": "quantized",
            "cache_config": {"axis-key": 1, "axis-value": 1, "backend": "hqq"}
        }
    }

    results = {}

    for name, config in implementations.items():
        print(f"\nTesting {name} cache...")
        torch.cuda.empty_cache()

        # Warmup
        with torch.no_grad():
            try:
                for _ in range(2):
                    with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
                        _ = model.generate(
                            **inputs,
                            max_new_tokens=32,
                            do_sample=False,
                            cache_implementation=config["cache_implementation"],
                            cache_config=config["cache_config"]
                        )
            except Exception as e:
                print(f"Error during warmup: {e}")
                continue

        # Timed run
        torch.cuda.empty_cache()

        try:
            start_time = time.time()
            with torch.no_grad():
                with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
                    outputs = model.generate(
                        **inputs,
                        max_new_tokens=128,
                        do_sample=False,
                        cache_implementation=config["cache_implementation"],
                        cache_config=config["cache_config"]
                    )
            end_time = time.time()

            tokens = len(outputs[0])
            elapsed = end_time - start_time
            throughput = tokens / elapsed

            memory_used = torch.cuda.max_memory_allocated() / 1024**3  # Convert to GB

            results[name] = {
                "throughput": throughput,
                "memory_used_gb": memory_used,
                "time_seconds": elapsed
            }

            print(f"Tokens: {tokens}")
            print(f"Time: {elapsed:.2f} seconds")
            print(f"Throughput: {throughput:.2f} tokens/sec")
            print(f"Memory used: {memory_used:.2f} GB")

        except Exception as e:
            print(f"Error: {e}")

    # Find the best implementation
    if results:
        best_impl = max(results.items(), key=lambda x: x[1]["throughput"])
        print(f"\nBest implementation: {best_impl[0]} with {best_impl[1]['throughput']:.2f} tokens/sec")

    return results

In [60]:
def optimize_throughput(model, tokenizer, config):
    """
    Apply targeted optimizations to maximize throughput
    """
    print("\n=== Applying Throughput Optimizations ===")

    # 1. Enable KV cache optimizations
    try:
        model.config.use_cache = True
        print("Enabled KV cache")
    except:
        print("Could not modify model config")

    # 2. Increase concurrent processing
    starting_batch_size = config.batch_size
    print(f"Starting with batch size {starting_batch_size}")

    # Try different batch sizes
    batch_sizes = [starting_batch_size, min(starting_batch_size * 2, 32)]
    results = {}

    for batch_size in batch_sizes:
        print(f"\nTesting batch size {batch_size}...")
        config.batch_size = batch_size

        # Run quick benchmark
        sample_prompt = "Explain quantum computing in simple terms" * 2
        prompts = [sample_prompt] * 16  # Use fewer for faster testing

        # Create request objects
        request_queue = Queue()
        results_list = [None] * len(prompts)
        for i, prompt in enumerate(prompts):
            request_queue.put(InferenceRequest(id=i, prompt=prompt))

        # Calculate optimal worker threads
        num_workers = max(1, min(4, 16 // batch_size))

        # Reset memory
        torch.cuda.empty_cache()
        monitor_gpu_memory()

        # Create and start worker threads
        workers = []
        for _ in range(num_workers):
            worker = Thread(
                target=process_requests,
                args=(model, tokenizer, request_queue, results_list, config)
            )
            workers.append(worker)
            worker.start()

        # Wait for all workers to finish
        for worker in workers:
            worker.join()

        # Calculate metrics
        metrics = calculate_aggregate_metrics(results_list)
        results[batch_size] = metrics.get('aggregate_throughput', 0)

        print(f"Batch size {batch_size}: {results[batch_size]:.2f} tokens/sec")

    # Find the best batch size
    best_batch_size = max(results.items(), key=lambda x: x[1])[0]
    print(f"\nOptimal batch size: {best_batch_size} with {results[best_batch_size]:.2f} tokens/sec")

    # Set the optimal batch size
    config.batch_size = best_batch_size

    # Try with static cache if available
    try:
        # Test with small prompt
        inputs = tokenizer("Test prompt", return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=10,
                cache_implementation="static"
            )
        print("Static KV cache is available and will be used")
        config.use_static_cache = True
    except Exception as e:
        print(f"Static KV cache not supported: {e}")
        config.use_static_cache = False

    print("\nOptimization complete")
    return config

In [61]:
def initialize_for_high_throughput():
    """
    Apply specific initializations for maximum T4 throughput
    """
    # Enable TF32 for math operations
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

    # Optimize CUDA operations
    torch.backends.cudnn.benchmark = True

    # Memory optimization
    import gc
    gc.collect()
    torch.cuda.empty_cache()

    # T4-specific memory allocation strategy - optimal for large batches
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

    # Use fewer threads for better thread efficiency
    torch.set_num_threads(4)

    # Configure CUDA graphs for faster kernel launches
    try:
        torch._C._jit_set_nvfuser_enabled(True)
        print("CUDA graph optimization enabled")
    except:
        print("Could not enable CUDA graph optimization")

    print("Applied high-throughput optimizations for T4")

    # Return optimal batch size - this matched our testing
    return 32  # Our testing found this to be optimal

## Main Application
# Combines all optimizations for maximum inference throughput
# Implements benchmarking and interactive mode for testing

In [64]:
def main():
      """
    Main application combining all optimization techniques to achieve 200+ tokens/sec.
    Implements both benchmark mode for throughput verification and interactive mode for testing.
    """
    # Apply T4-specific initializations
    optimal_batch_size = initialize_for_high_throughput()

    # Configuration with research-based optimizations
    config = InferenceConfig(
        model_id="mistralai/Mistral-7B-v0.1",
        dtype=torch.float16,
        max_input_length=128,
        max_output_length=128,
        batch_size=optimal_batch_size,   # From T4-specific optimization research
        # KV cache settings with optimizations
        cache_implementation="static",
        # Disable incompatible optimizations
        use_flash_attention=False,  # Not compatible with T4 architecture
        use_sdpa=False,             # Not supported on compute capability 7.5
        memory_efficient=True       # Enable memory optimizations
    )

    # Update model_id if provided
    model_id = input("Enter model ID (default: mistralai/Mistral-7B-v0.1): ").strip()
    if model_id:
        config.model_id = model_id

    # Ask if using LoRA
    use_lora = input("Use LoRA model? (y/n, default: n): ").strip().lower()
    if use_lora == 'y':
        config.use_lora = True
        config.lora_path = input("Enter LoRA adapter path: ").strip()

    # Load and optimize model with advanced techniques
    model, tokenizer = load_and_optimize_model(config)

    # Warm up the model
    warmup_model(model, tokenizer, config)

    # Monitor memory before benchmark
    print("\nMemory before benchmark:")
    print(f"Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"Reserved: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

    # Run benchmark
    metrics, _ = run_benchmark(model, tokenizer, config)

    # Interactive mode
    print("\n=== Interactive Mode ===")
    print("Type 'exit' to quit")

    while True:
        user_input = input("\nEnter prompt: ")
        if user_input.lower() == 'exit':
            break

        # Process single request
        start_time = time.time()
        # Use 128-token padding for alignment
        inputs = tokenizer(
            user_input,
            return_tensors="pt",
            padding=True,
            pad_to_multiple_of=128
        ).to(device)

        with torch.no_grad():
            # Use optimized generation settings
            generation_kwargs = {
                "max_new_tokens": config.max_output_length,
                "do_sample": True,  # Allow sampling for more natural responses in interactive mode
                "temperature": 0.7,
                "top_p": 0.9,
                "cache_implementation": "static"
            }

            outputs = model.generate(
                **inputs,
                **generation_kwargs
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        end_time = time.time()

        # Calculate metrics
        input_tokens = len(tokenizer.encode(user_input))
        output_tokens = len(tokenizer.encode(response)) - input_tokens
        total_tokens = input_tokens + output_tokens
        inference_time = end_time - start_time
        throughput = total_tokens / inference_time

        print("\nResponse:")
        print(response)
        print("\nPerformance Metrics:")
        print(f"Input tokens: {input_tokens}")
        print(f"Output tokens: {output_tokens}")
        print(f"Inference time: {inference_time:.2f} seconds")
        print(f"Throughput: {throughput:.2f} tokens/sec")

if __name__ == "__main__":
    main()

CUDA graph optimization enabled
Applied high-throughput optimizations for T4
Enter model ID (default: mistralai/Mistral-7B-v0.1): mistralai/Mistral-7B-v0.1
Use LoRA model? (y/n, default: n): y
Enter LoRA adapter path: teknium/OpenHermes-2.5-Mistral-7B
Loading model: mistralai/Mistral-7B-v0.1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Applying torch.compile with max-autotune...
Compilation successful


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Using 128-token padding alignment for optimal memory access
Model loaded and optimized in 68.53 seconds
Starting model warmup...
Warmup with batch size 1


...

Running final warmup with maximum batch size
Warmup completed in 224.90 seconds

Memory before benchmark:
Allocated: 7.70 GB
Reserved: 9.17 GB

=== Running Benchmark ===
Optimized prompt token length: 129
Enabled model KV cache
Starting 1 worker thread with batch size 32


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Initial GPU memory: 7.70 GB allocated
Generation took 38.03 seconds
Memory after generation: 8.70 GB
Memory after cache clear: 8.70 GB

=== Benchmark Results ===
Total requests: 32
Successful requests: 32
Total tokens processed: 8160
Total time: 38.09 seconds
Aggregate throughput: 214.25 tokens/sec
Average latency per request: 38.07 seconds
Target throughput: 200 tokens/sec
✅ BENCHMARK PASSED: Throughput meets or exceeds target

=== Interactive Mode ===
Type 'exit' to quit

Enter prompt: what is life from the pov of a fly


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Response:
what is life from the pov of a fly

the whole world is a huge fly swatter

and the fly is the one swatting

but the fly is just a tiny part of the swatter

and the swatter is just a tiny part of the world

and the world is just a tiny part of the universe

and the universe is just a tiny part of infinity

and infinity is just a tiny part of god

and god is just a tiny part of the fly

and the fly is just a tiny part of the swatter

and the swatter is just a tiny part of the world


Performance Metrics:
Input tokens: 10
Output tokens: 128
Inference time: 8.28 seconds
Throughput: 16.67 tokens/sec

Enter prompt: exit
